### Packages

In [4]:
# pip install webdriver-manager


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
# pip install selenium
# pip install webdriver-manager

# Packages for webscraping
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

#Package to handle dataframes
import pandas as pd

#Package for progress bar
from tqdm import tqdm

#### Configure webdriver to use Chrome browser

In [6]:
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

[WDM] - Downloading: 100%|██████████| 6.68M/6.68M [00:02<00:00, 2.57MB/s]


#### ID selection (phenotypes)

In [2]:
### Add
# 6177, 6153, 1588, 1578, 1608, 5364, 1568, 1598, 904, 6138, 20116, 1289, 1309, 2040, 20023, 1200, 1498



# Select UKBB ids of interest
Biomarker_CAL = [30600,30610,30620,30630,30640,30650,30840,2744,20022,3148,21001,23105,23100,30680,30690,30700,
              30710,30720,4079,30150,20016,3062,30730,30740,46,47,30750,30760,102,50,30770,30780,30790,30120,
              30050,30130,20127,30140,30810,30080,30010,30250,30830,4080,30850,30870,30860,30880,30670,30890,
              30000,21002,48,49, #Attention 46,47,48,49 ratio in Auwerx Phenotypes (57 Phenos, some moved)
                 22192, 30800, 22426] # Remove 51, 22433

# Need to calculate and mean grip strength  WHR, WHR adjusted for BMI
# 2714, 3581, 2395 and 2375 moved to Life history

print("Nb of biomarkers:", len(Biomarker_CAL))

Disease_NTL = []

print("Nb of diseases:", len(Disease_NTL))

Lifestyle_SML = [1160, 20117, 1558, 100940, 1369, 1408, 1528, 1110, 2237, 2644, 189, 845,
                738, 2267] # 20118 might wanna remove Urban/Rural

#Remove 21067, 1031, 20447, 20433

print("Nb of lifstyle phenotypes:", len(Lifestyle_SML))

Life_history_SML = [40007, 1807, 3526, 2754, 2764, 2714, 3581, 2395, 2375, 40008, 21003, 2217, 2207, 2734]

#Remove 3393

# Difference between 2714 and 3581 should be calculated (female repoductive period)

print("Nb of life history phenotypes:", len(Life_history_SML))

print("Total:", len(Biomarker_CAL) + len(Disease_NTL) + len(Lifestyle_SML) + len (Life_history_SML))
print("Check no recurrency:", len(set().union(Biomarker_CAL,Disease_NTL,Lifestyle_SML,Life_history_SML)))

Nb of biomarkers: 57
Nb of diseases: 0
Nb of lifstyle phenotypes: 14
Nb of life history phenotypes: 14
Total: 85
Check no recurrency: 85


### Webscraping function

In [7]:
def UKBB_to_DF(ukbbidlist):
    # Initiate empty lists to store data of interest
    Description=[]
    Category = []
    ValueType=[]
    Nbpart = []
    Sex=[] 
    Mean=[]
    Median=[]
    Stdv=[]

    # 
    for ukbbid in tqdm(ukbbidlist):
        driver = webdriver.Chrome(service=s)
        driver.get("https://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=" + str(ukbbid))
    # Search by XPATH (put XPATH between """___""")
        Description.append(driver.find_element(by=By.XPATH, value="""//*[@id="main"]/table[1]/tbody/tr[1]/td[2]""").text)
        Category.append(driver.find_element(by=By.XPATH, value="""//*[@id="main"]/table[1]/tbody/tr[2]/td[2]/a[1]""").text)
        ValueType.append(driver.find_element(by=By.XPATH, value="""//*[@id="main"]/table[2]/tbody/tr[1]/td[5]""").text)
        Nbpart.append(int(driver.find_element(by=By.XPATH, value="""//*[@id="main"]/table[2]/tbody/tr[1]/td[2]""").text.replace(",", "")))
        Sex.append(driver.find_element(by=By.XPATH, value="""//*[@id="main"]/table[2]/tbody/tr[1]/td[8]""").text)
    # Search by text element (write search query e.g. "Mean = " and remove by replacing string)
        try:
            Mean.append(driver.find_element(By.XPATH, value = ("//*[contains(text(),'Mean = ')]")).text.replace("Mean = ", ""))
        except NoSuchElementException:
            Mean.append("NA")
    
        try:    
            Median.append(driver.find_element(By.XPATH, value = ("//*[contains(text(),'Median')]/following::td")).text)
        except NoSuchElementException:
            Median.append("NA")

        try:
            Stdv.append(driver.find_element(By.XPATH, value = ("//*[contains(text(),'Std.dev = ')]")).text.replace("Std.dev = ", ""))
        except NoSuchElementException:
            Stdv.append("NA")

        driver.close()
    # Write dataframe
    df = pd.DataFrame({'FieldID':ukbbidlist,'Description':Description, "Category":Category,
                       "Value_Type":ValueType, "Nb_Participants": Nbpart,
                       'Sex':Sex,"Mean":Mean, "Median": Median, "SD":Stdv}) 
    
    return(df)

### Get data as dataframe

In [7]:
df1 = UKBB_to_DF(Biomarker_CAL)
df2 = UKBB_to_DF(Disease_NTL)
df3 = UKBB_to_DF(Lifestyle_SML)
df4 = UKBB_to_DF(Life_history_SML)

100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [01:28<00:00,  1.54s/it]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:21<00:00,  1.49s/it]


In [9]:
frames = [df1, df2, df3, df4]
df = pd.concat(frames)
df.head(10)

,FieldID,Description,Category,Value_Type,Nb_Participants,Sex,Mean,Median,SD
0,30600.0,Albumin,Blood biochemistry,"Continuous, g/L",432261.0,Both sexes,45.2248,45.21,2.62932
1,30610.0,Alkaline phosphatase,Blood biochemistry,"Continuous, U/L",470774.0,Both sexes,83.7016,80.4,26.3751
2,30620.0,Alanine aminotransferase,Blood biochemistry,"Continuous, U/L",470573.0,Both sexes,23.526,20.13,14.1116
3,30630.0,Apolipoprotein A,Blood biochemistry,"Continuous, g/L",429714.0,Both sexes,1.53926,1.512,0.270897
4,30640.0,Apolipoprotein B,Blood biochemistry,"Continuous, g/L",468429.0,Both sexes,1.03169,1.017,0.238221
5,30650.0,Aspartate aminotransferase,Blood biochemistry,"Continuous, U/L",469024.0,Both sexes,26.2417,24.4,10.6484
6,30840.0,Total bilirubin,Blood biochemistry,"Continuous, umol/L",468801.0,Both sexes,9.11927,8.06,4.42528
7,2744.0,Birth weight of first child,Female-specific factors,"Integer, pounds",221459.0,Females only,7.0114,7,1.21556
8,20022.0,Birth weight,Early life factors,"Continuous, Kg",280200.0,Both sexes,3.32018,3.32,0.667736
9,3148.0,Heel bone mineral density (BMD),Bone-densitometry of heel,"Continuous, g/cm2",278904.0,Both sexes,0.543541,0.53,0.138914


In [10]:
df.to_csv('../', index=False, encoding='utf-8')
df.to_excel('../', index=False, encoding='utf-8')

### From TL_metadata file

In [23]:
### Load dataframe
df_metadata = pd.read_csv(r'../TL_metadata.csv')
### Get list of FieldIDs
fieldids = list(df_metadata["FieldID"])
fieldids = [x for x in fieldids if str(x) != 'nan'] # Remove NaN
fieldids = [int(i) for i in fieldids] # Convert float to int

In [24]:
### Extract information
df = UKBB_to_DF(fieldids)

100%|██████████| 90/90 [03:13<00:00,  2.15s/it]


In [25]:
df

,FieldID,Description,Category,Value_Type,Nb_Participants,Sex,Mean,Median,SD
0,21003,Age when attended assessment centre,Reception,"Integer, years",502411,Both sexes,57.5481,59,8.44123
1,40008,Age at cancer diagnosis,Cancer register,Continuous,118980,Both sexes,60.389,63,12.4027
2,40007,Age at death,Death register,"Continuous, years",37897,Both sexes,70.1278,71.4,7.46689
3,845,Age completed full time education,Education,"Integer, years",337352,Both sexes,16.7349,16,2.33829
4,2754,Age at first live birth,Female-specific factors,"Integer, years",184998,Females only,25.4356,25,4.62531
...,...,...,...,...,...,...,...,...,...
85,48,Waist circumference,Body size measures,"Continuous, cm",500318,Both sexes,90.1805,90,13.4194
86,1528,Water intake,Diet,"Integer, glasses/day",501532,Both sexes,2.86044,2,2.2286
87,30000,White blood cell (leukocyte) count,Blood count,"Continuous, 10^9 cells/Litre",479256,Both sexes,6.88107,6.65,2.13168
88,21002,Weight,Body size measures,"Continuous, Kg",499731,Both sexes,77.8425,76.2,15.8785


In [29]:
df.to_csv('../', index=False, encoding='utf-8')
df.to_excel('../', index=False, encoding='utf-8')